# Woche 7: Data Cleaning - Übung am eigenen Projekt

**Ziel dieser Übung:** Nachdem Sie die wichtigsten Aspekte des Data Cleanings am Airbnb-Beispiel kennengelernt haben, wenden Sie dieses Wissen Schritt für Schritt auf Ihren eigenen Datensatz an.

**Arbeitsweise:**
- Arbeiten Sie die Aufgaben nacheinander durch
- Nutzen Sie die Code-Zellen für Ihre Implementierung
- Orientieren Sie sich an den Beispielen aus dem Airbnb-Notebook
- Das bereinigte Dataset speichern Sie am Ende ab

---
## 1. Daten einlesen und Bibliotheken importieren

**Aufgabe:** Importieren Sie die notwendigen Bibliotheken und laden Sie Ihren Datensatz.

**Hinweise:**
- Importieren Sie: `pandas`, `numpy`, `matplotlib.pyplot`, `seaborn`
- Setzen Sie einen Zufallsseed für Reproduzierbarkeit
- Laden Sie Ihren CSV-Datensatz mit `pd.read_csv()`

In [2]:
# Bibliotheken importieren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno


# Zufallsseed setzen
np.random.seed(42)


# Datensatz einlesen
# df = pd.read_csv('ihr_datensatz.csv')
df = pd.read_csv('data/economy.csv')


C:\Users\thoma\AppData\Local\Temp\ipykernel_41284\962934862.py:15: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/economy.csv')


---
## 2. Ersten Überblick verschaffen

**Aufgabe:** Verschaffen Sie sich einen ersten Überblick über Ihren Datensatz.

**Was Sie prüfen sollten:**
- Wie viele Zeilen und Spalten hat der Datensatz?
- Welche Spalten gibt es und welche Datentypen haben sie?
- Wie sehen die ersten Zeilen aus?

In [3]:
# Dimensionen des Datensatzes
print(df.shape)

# Erste Zeilen anzeigen
print(df.head())

# Informationen zu Spalten und Datentypen
print(df.info())

(43234, 99)
         date  match_id  event_id         team_1         team_2 best_of  \
0  2020-03-01   2339402      4901             G2  Natus Vincere       5   
1  2020-03-01   2339402      4901             G2  Natus Vincere       5   
2  2020-03-01   2339402      4901             G2  Natus Vincere       5   
3  2020-02-29   2339401      4901  Natus Vincere       Astralis       3   
4  2020-02-29   2339401      4901  Natus Vincere       Astralis       3   

     _map t1_start t2_start    1_t1  ...  21_winner  22_winner  23_winner  \
0    Nuke        t       ct  4350.0  ...        NaN        NaN        NaN   
1   Dust2       ct        t  3900.0  ...        2.0        1.0        1.0   
2  Mirage        t       ct  4150.0  ...        NaN        NaN        NaN   
3   Dust2        t       ct  4150.0  ...        1.0        NaN        NaN   
4    Nuke       ct        t  4200.0  ...        1.0        NaN        NaN   

   24_winner  25_winner  26_winner  27_winner  28_winner  29_winner  30_wi

---
## 3. Fehlende Werte identifizieren

**Aufgabe:** Untersuchen Sie Ihren Datensatz auf fehlende Werte.

**Was Sie tun sollten:**
- Zählen Sie fehlende Werte pro Spalte
- Berechnen Sie den Prozentsatz fehlender Werte
- Visualisieren Sie fehlende Werte (optional: Heatmap)

In [4]:
# Fehlende Werte zählen
#df.isnull().sum()
df.isna().sum()

# Prozentsatz fehlender Werte berechnen
df.isna().mean()

# Optional: Visualisierung mit Heatmap
#msno.heatmap(df)

date         0.000000
match_id     0.000000
event_id     0.000000
team_1       0.000000
team_2       0.000000
               ...   
26_winner    0.478836
27_winner    0.561734
28_winner    0.646413
29_winner    0.731161
30_winner    0.817297
Length: 99, dtype: float64

---
## 4. Fehlende Werte behandeln

**Aufgabe:** Entscheiden Sie für jede Spalte mit fehlenden Werten, wie Sie damit umgehen.

**Mögliche Strategien:**
- Zeilen löschen (bei wenigen fehlenden Werten)
- Spalten löschen (bei sehr vielen fehlenden Werten)
- Fehlende Werte imputieren:
  - Numerische Spalten: Median oder Mittelwert
  - Kategoriale Spalten: Modus oder neue Kategorie

**Dokumentieren Sie Ihre Entscheidungen!**

In [5]:
round_data = []

for idx, row in df.iterrows():
    match_id = row['match_id']
    map_name = row['_map']
    team1_side = row['t1_start']
    team2_side = row['t2_start']
    
    for round_num in range(1, 31):
        winner_col = f'{round_num}_winner'
        team1_equip_col = f'{round_num}_t1'
        team2_equip_col = f'{round_num}_t2'
        
        if pd.notna(row[winner_col]):
            if round_num <= 15:
                current_team1_side = team1_side
                current_team2_side = team2_side
            else:
                current_team1_side = team2_side
                current_team2_side = team1_side
            
            t1_equip = int(row[team1_equip_col])
            t2_equip = int(row[team2_equip_col])
            equip_diff = t1_equip - t2_equip
            
            round_entry = {
                'match_id': int(match_id),
                'map': map_name,
                'round': int(round_num),
                'team1_equip': t1_equip,
                'team2_equip': t2_equip,
                'equipment_diff': equip_diff,
                'team1_side': current_team1_side,
                'team2_side': current_team2_side,
                'winner': int(row[winner_col])
            }
            
            round_data.append(round_entry)

df_rounds = pd.DataFrame(round_data)
df_rounds.to_csv("data/economy_rounds_cleaned.csv", index=False)

In [6]:
# Überprüfung: Sind alle fehlenden Werte behandelt?
df_rounds.isnull().sum()
df_rounds.isna().sum()

match_id          0
map               0
round             0
team1_equip       0
team2_equip       0
equipment_diff    0
team1_side        0
team2_side        0
winner            0
dtype: int64

---
## 5. Duplikate identifizieren und entfernen

**Aufgabe:** Prüfen Sie, ob Ihr Datensatz doppelte Zeilen enthält.

**Was Sie tun sollten:**
- Zählen Sie die Anzahl doppelter Zeilen
- Entfernen Sie Duplikate (falls vorhanden)
- Überprüfen Sie die neue Anzahl der Zeilen

In [7]:
# Dimensionen prüfen
print(df_rounds.shape)

# Anzahl doppelter Zeilen
print(df_rounds.duplicated().sum())

# Duplikate entfernen
df_rounds = df_rounds.drop_duplicates()

# Neue Dimensionen prüfen
df_rounds.shape


(1095217, 9)
0


(1095217, 9)

---
## 6. Datentypen überprüfen und anpassen

**Aufgabe:** Stellen Sie sicher, dass alle Spalten die richtigen Datentypen haben.

**Was Sie prüfen sollten:**
- Sind numerische Spalten als `int` oder `float` kodiert?
- Sind kategoriale Spalten als `object` oder `category` kodiert?
- Müssen Datentypen konvertiert werden?

In [8]:
# Datentypen anzeigen


# Datentypen konvertieren (falls nötig)
# df['spaltenname'] = df['spaltenname'].astype('int')
# df['spaltenname'] = df['spaltenname'].astype('category')
df_rounds.dtypes



match_id           int64
map               object
round              int64
team1_equip        int64
team2_equip        int64
equipment_diff     int64
team1_side        object
team2_side        object
winner             int64
dtype: object

---
## 7. Ausreißer identifizieren

**Aufgabe:** Identifizieren Sie Ausreißer in numerischen Spalten.

**Methoden:**
- Visualisierung mit Boxplots
- IQR-Methode (Interquartile Range)
- Statistische Analyse (describe)

**Wichtig:** Entscheiden Sie für jeden Ausreißer, ob er:
- Ein Fehler ist (→ entfernen oder korrigieren)
- Ein echter extremer Wert ist (→ behalten)

In [ ]:
#Numerische Spalten auswählen
#numeric_cols = df_rounds.select_dtypes(include=['int64', 'float64']).columns


# Statistische Übersicht


# Boxplots für numerische Spalten erstellen


Index(['match_id', 'round', 'team1_equip', 'team2_equip', 'equipment_diff',
       'winner'],
      dtype='object')


In [10]:
# Ausreißer mit IQR-Methode identifizieren (Beispiel für eine Spalte)
# Q1 = df['spaltenname'].quantile(0.25)
# Q3 = df['spaltenname'].quantile(0.75)
# IQR = Q3 - Q1
# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR

# outliers = df[(df['spaltenname'] < lower_bound) | (df['spaltenname'] > upper_bound)]


---
## 8. Ausreißer behandeln

**Aufgabe:** Behandeln Sie die identifizierten Ausreißer entsprechend Ihrer Analyse.

**Mögliche Strategien:**
- Ausreißer entfernen (wenn sie Fehler sind)
- Ausreißer behalten (wenn sie valide sind)
- Ausreißer begrenzen (Capping/Flooring)

**Dokumentieren Sie Ihre Entscheidungen!**

In [11]:
# Beispiel: Ausreißer entfernen
# df = df[(df['spaltenname'] >= lower_bound) & (df['spaltenname'] <= upper_bound)]


# Beispiel: Capping anwenden
# df['spaltenname'] = df['spaltenname'].clip(lower=lower_bound, upper=upper_bound)


---
## 9. Inkonsistenzen beheben

**Aufgabe:** Suchen Sie nach Inkonsistenzen in kategorialen Spalten.

**Was Sie prüfen sollten:**
- Unterschiedliche Schreibweisen (z.B. "ja", "Ja", "JA")
- Leerzeichen am Anfang oder Ende
- Tippfehler
- Unerwartete Kategorien

In [12]:
# Kategoriale Spalten auswählen
# categorical_cols = df.select_dtypes(include=['object', 'category']).columns


# Einzigartige Werte pro kategorialer Spalte anzeigen
# for col in categorical_cols:
#     print(f"\n{col}:")
#     print(df[col].value_counts())


In [13]:
# Inkonsistenzen beheben
# Beispiel: Leerzeichen entfernen und in Kleinbuchstaben umwandeln
# df['spaltenname'] = df['spaltenname'].str.strip().str.lower()


# Beispiel: Werte ersetzen
# df['spaltenname'] = df['spaltenname'].replace({'alter_wert': 'neuer_wert'})


---
## 10. Finale Überprüfung

**Aufgabe:** Führen Sie eine finale Qualitätskontrolle durch.

**Checkliste:**
- ✓ Keine fehlenden Werte (oder bewusst belassen)
- ✓ Keine Duplikate
- ✓ Korrekte Datentypen
- ✓ Ausreißer behandelt
- ✓ Inkonsistenzen behoben
- ✓ Datensatz ist bereit für die Analyse

In [14]:
# Finale Übersicht
print("Finale Dimensionen:")
print(df_rounds.shape)

print("\nFehlende Werte:")
print(df_rounds.isnull().sum())

print("\nDatentypen:")
print(df_rounds.dtypes)

print("\nErste Zeilen des bereinigten Datensatzes:")
print(df_rounds.head())

Finale Dimensionen:
(1095217, 9)

Fehlende Werte:
match_id          0
map               0
round             0
team1_equip       0
team2_equip       0
equipment_diff    0
team1_side        0
team2_side        0
winner            0
dtype: int64

Datentypen:
match_id           int64
map               object
round              int64
team1_equip        int64
team2_equip        int64
equipment_diff     int64
team1_side        object
team2_side        object
winner             int64
dtype: object

Erste Zeilen des bereinigten Datensatzes:
   match_id   map  round  team1_equip  team2_equip  equipment_diff team1_side  \
0   2339402  Nuke      1         4350         4250             100          t   
1   2339402  Nuke      2         1100        20250          -19150          t   
2   2339402  Nuke      3        22100        24600           -2500          t   
3   2339402  Nuke      4         9350        29300          -19950          t   
4   2339402  Nuke      5        25750        30650       

---
## 11. Bereinigten Datensatz speichern

**Aufgabe:** Speichern Sie Ihren bereinigten Datensatz als CSV-Datei.

**Wichtig:** Dieser bereinigte Datensatz wird in den kommenden Wochen für Visualisierung und Machine Learning verwendet!

In [15]:
# Bereinigten Datensatz speichern
df_rounds.to_csv('data/economy_rounds_cleaned.csv', index=False)

print("Bereinigter Datensatz wurde gespeichert!")

Bereinigter Datensatz wurde gespeichert!


---
## Reflexion

**Dokumentieren Sie Ihre Arbeit:**

Beantworten Sie folgende Fragen in einer Markdown-Zelle:

1. Welche Hauptprobleme hatte Ihr ursprünglicher Datensatz?
2. Welche Bereinigungsschritte waren am wichtigsten?
3. Wie viele Zeilen/Spalten haben Sie entfernt und warum?
4. Welche Herausforderungen gab es und wie haben Sie diese gelöst?
5. Ist Ihr Datensatz jetzt bereit für die Analyse?

### Ihre Reflexion:

1. **Hauptprobleme:**
   - Der Datensatz war im "Wide Format" mit 99 Spalten sehr unübersichtlich und schwer zu analysieren.
   - Es gab extrem viele fehlende Werte (NaN) für Runden, die gar nicht gespielt wurden (wenn ein Match 16:5 endete).
   - Die Team-Seiten (CT/T) waren nur für den Start angegeben, änderten sich aber im Spielverlauf.

2. **Wichtigste Schritte:**
   - Die Transformation von eine Zeile als Match in eine Zeile als Runde wurde umgesetzt.
   - Das Herausfiltern der nicht gespielten Runden.
   - Die korrekte Implementierung der Logik für den Seitenwechsel nach Runde 15.

3. **Entfernte Daten:**
   - Alle Runden ohne Gewinner (nicht gespielte Runden) wurden entfernt.
   - Die Spaltenanzahl wurde von 99 auf die 8 relevanten Features reduziert (match_id, map, round, equip, side, winner).

4. **Herausforderungen:**
   - Den Seitenwechsel (CT <-> T) nach der 15. Runde programmtechnisch korrekt abzubilden, war die kniffligste   Stelle.
   - Sicherzustellen, dass beim Umformen keine Daten durcheinandergeraten.

5. **Bereitschaft für Analyse:**
   - Der Datensatz sollte jetzt sauber sein und keine fehlenden Werte mehr enthalten.